In [2]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

#fitting regression line
def fit_regression_line(x, y):
    x = np.array(x).reshape(-1, 1)
    y = np.array(y)
    model = LinearRegression().fit(x, y)
    y_pred = model.predict(x)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x.flatten(), y)
    return model, y_pred, slope, intercept, r_value, p_value, std_err, len(x)

In [4]:

# list of file names to re-plot
file_names = [
    "40_B",
    "42_A",
    "42_B",
    "45_A",
    "45_B",
    "45_C",
    "47_A",
    "47_B",
    "47_C"
]

# folder that contains the rounded plot data files
rounded_data_dir = 'rounded_data_files'

# folder of original plot images
image_dir = 'plot_images'

# folder to save the re-plotted figures
output_dir = 'replotted_figures_rounded_with_regression_lines'
os.makedirs(output_dir, exist_ok=True)

# dicts for scales, ticks, and axis labels
scales = {
    "40_B": ((2.5, 6.5), (2.5, 22.5)),
    "42_A": ((2.5, 6.5), (0.5, 10.25)),
    "42_B": ((2.5, 6.5), (0, 8)),
    "45_A": ((2.5, 6.5), (-300, 150)),
    "45_B": ((2.5, 6.5), (25, 300)),
    "45_C": ((2.5, 6.5), (-32, 90)),
    "47_A": ((2.5, 6.5), (-25, 25)),
    "47_B": ((2.5, 6.5), (8.5, 65)),
    "47_C": ((2.5, 6.5), (0, 0.7))
}

# xticks are same for all the plots, no need to have them here. 
# xticks = {
# }

yticks = {
    "40_B": [5, 10, 15, 20],
    "42_A": [2, 4, 6, 8, 10],
    "42_B": [2, 4, 6],
    "45_A": [-200, -100, 0, 100],
    "45_B": [50, 100, 150, 200, 250],
    "45_C":  [-20, 0, 20, 40, 60, 80],
    "47_A": [-20, -10, 0, 10, 20],
    "47_B": [10, 20, 30, 40, 50, 60],
    "47_C": [0.2, 0.4, 0.6]
}

axis_labels = {
    "40_B": ('Mediolateral Position (mm)', 'Response Latency (ms)'),
    "42_A": ('Mediolateral Position (mm)', 'Best Frequency (kHz)'),
    "42_B": ('Mediolateral Position (mm)', 'Frequency Tuning Width (kHz)'),
    "45_A": ('Mediolateral Position (mm)', 'Best ITD (us)'),
    "45_B": ('Mediolateral Position (mm)', 'ITD tuning Width (us)'),
    "45_C": ('Mediolateral Position (mm)', 'Sidepeak Supression (%)'),
    "47_A": ('Mediolateral Position (mm)', 'Best ILD (dB)'),
    "47_B": ('Mediolateral Position (mm)', 'ILD tuning Width (dB)'),
    "47_C": ('Mediolateral Position (mm)', 'ILD Symmetry Index')
}

In [8]:

# looping through each plot file to create re-plotted figures with regression lines
for plot_file in file_names:
    x_points = []
    y_points = []

    file_path = os.path.join(rounded_data_dir, f'{plot_file}_rounded.csv')
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            x, y = map(float, row)
            x_points.append(x)
            y_points.append(y)

    x_scale, y_scale = scales[plot_file]
    xticks_vals = [3, 4, 5, 6]
    yticks_vals = yticks[plot_file]
    x_label, y_label = axis_labels[plot_file]

    # fitting regression line
    model, y_pred, slope, intercept, r_value, p_value, std_err, n = fit_regression_line(x_points, y_points)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4.5))

    # plotting original data with regression line
    ax1.scatter(x_points, y_points, edgecolor='black', facecolor='none', s=15)
    ax1.plot(x_points, y_pred, color='red', linestyle='--')
    ax1.set_xlim(x_scale)
    ax1.set_ylim(y_scale)
    ax1.set_xticks(xticks_vals)
    ax1.set_yticks(yticks_vals)
    ax1.set_title(f'Rounded Data with Regression Line ({plot_file})')
    ax1.set_xlabel(x_label)
    ax1.set_ylabel(y_label)

    stats_text = (f'r: {r_value:.3f}\np: {p_value:.4f}\nn: {n}')
    ax1.text(0.05, 0.95, stats_text, transform=ax1.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    
    #displaying the original plot here
    original_plot_path = os.path.join(image_dir, f'{plot_file}.png')
    original_img = plt.imread(original_plot_path)
    ax2.imshow(original_img)
    ax2.axis('off')
    ax2.set_title(f'Original Plot ({plot_file})')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{plot_file}_regression.png'))
    plt.show()
    plt.close()

print("Success!!!")